In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

In [2]:
from sklearn.datasets import fetch_california_housing
california = fetch_california_housing()

In [3]:
df = pd.DataFrame(california.data, columns=california.feature_names)
df["Target"] = california.target
df.tail()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Target
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847
20639,2.3886,16.0,5.254717,1.162264,1387.0,2.616981,39.37,-121.24,0.894


In [4]:
scaler = StandardScaler()
scaler.fit(df.values[:, :-1])
df.values[:, :-1] = scaler.transform(df.values[:, :-1])

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
data = torch.from_numpy(df.values).float()
data.shape

torch.Size([20640, 9])

In [7]:
x = data[:, :-1]
y = data[:, -1:]

print(x.shape, y.shape)

torch.Size([20640, 8]) torch.Size([20640, 1])


In [8]:
n_epochs = 100
batch_size = 256
print_interval = 200
learning_rate = 1e-2

In [9]:
model = nn.Sequential(
    nn.Linear(x.size(-1), 6),
    nn.LeakyReLU(),
    nn.Linear(6, 5),
    nn.LeakyReLU(),
    nn.Linear(5, 4),
    nn.LeakyReLU(),
    nn.Linear(4, 3),
    nn.LeakyReLU(),
    nn.Linear(3, y.size(-1)),
)

model

Sequential(
  (0): Linear(in_features=8, out_features=6, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=6, out_features=5, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=5, out_features=4, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=4, out_features=3, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=3, out_features=1, bias=True)
)

In [10]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [11]:
for i in range(n_epochs):
    indices = torch.randperm(x.size(0))
    x_ = torch.index_select(x, dim=0, index=indices)
    y_ = torch.index_select(y, dim=0, index=indices)

    x_ = x_.split(batch_size, dim=0)
    y_ = y_.split(batch_size, dim=0)

    y_hat=[]
    total_loss = 0

    for x_i, y_i in zip(x_, y_):
      y_hat_i = model(x_i)
      loss = F.mse_loss(y_hat_i, y_i)

      optimizer.zero_grad()
      loss.backward()

      optimizer.step()

      total_loss += float(loss)
      y_hat += [y_hat_i]

    total_loss = total_loss / len(x_)
    if (i + 1) % print_interval != 0:
      print('Epoch %d: loss=%.3f' % (i + 1, total_loss))
    

y_hat = torch.cat(y_hat, dim=0)
y = torch.cat(y_,dim=0)
print('Training is finished!')

Epoch 1: loss=2.055
Epoch 2: loss=1.359
Epoch 3: loss=1.334
Epoch 4: loss=1.332
Epoch 5: loss=1.331
Epoch 6: loss=1.332
Epoch 7: loss=1.332
Epoch 8: loss=1.332
Epoch 9: loss=1.333
Epoch 10: loss=1.332
Epoch 11: loss=1.332
Epoch 12: loss=1.332
Epoch 13: loss=1.332
Epoch 14: loss=1.332
Epoch 15: loss=1.331
Epoch 16: loss=1.332
Epoch 17: loss=1.331
Epoch 18: loss=1.331
Epoch 19: loss=1.331
Epoch 20: loss=1.332
Epoch 21: loss=1.331
Epoch 22: loss=1.331
Epoch 23: loss=1.331
Epoch 24: loss=1.331
Epoch 25: loss=1.332
Epoch 26: loss=1.332
Epoch 27: loss=1.331
Epoch 28: loss=1.331
Epoch 29: loss=1.332
Epoch 30: loss=1.331
Epoch 31: loss=1.331
Epoch 32: loss=1.331
Epoch 33: loss=1.331
Epoch 34: loss=1.332
Epoch 35: loss=1.332
Epoch 36: loss=1.332
Epoch 37: loss=1.331
Epoch 38: loss=1.331
Epoch 39: loss=1.331
Epoch 40: loss=1.332
Epoch 41: loss=1.331
Epoch 42: loss=1.331
Epoch 43: loss=1.331
Epoch 44: loss=1.332
Epoch 45: loss=1.331
Epoch 46: loss=1.332
Epoch 47: loss=1.331
Epoch 48: loss=1.332
E